In [1]:
import json
import os
import sys

import cv2
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import ttach as tta
from aim_target.utils import compute_accuracy

with open("../configs/config.json", "r") as f:
    config = json.load(f)

model = torch.load(
    f"../{config['checkpoints']}/{config['image_size']}_{config['model']}.pt"
)
model.to("cuda")
model.eval();

tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.hflip_transform())

In [2]:
test_data = []
test_target = []
for i in os.listdir("../data/public_test_v2/"):
    for j in os.listdir(f"../data/public_test_v2/{i}"):
        if i == "human":
            test_target.append(0)
        elif i == "target_1human":
            test_target.append(1)
        elif i == "target_2laser":
            test_target.append(2)
        elif i == "target_3gun":
            test_target.append(3)
        elif i == "target_4tank":
            test_target.append(4)
            
        test_data.append(f"../data/public_test_v2/{i}/{j}")

In [3]:
fin_outputs = []
with torch.no_grad():
    for i in tqdm(test_data, total=len(test_data)):
        try:
            image = cv2.imread(i)
            image = cv2.resize(image, (config["image_size"], config["image_size"]))
            image = torch.tensor(np.moveaxis(image, -1, 0), dtype=torch.float).unsqueeze(0)
            image = image.to(config["device"])

            outputs = model(image)
            outputs = outputs.squeeze(1)
            outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
            fin_outputs.extend(outputs.tolist())
        except:
            print(i)

100%|██████████| 1942/1942 [00:29<00:00, 65.15it/s]


In [4]:
compute_accuracy(test_target, fin_outputs)

avg_acc 0.8326467559217302, per_class_acc human:0.61 target_human:0.9916666666666667 target_laser:0.9944444444444445 target_gun:0.9710743801652892 target_tank:1.0
score 40.638891439649015


(0.8326467559217302,
 [0.61, 0.9916666666666667, 0.9944444444444445, 0.9710743801652892, 1.0])

In [5]:
pd.DataFrame({"target": fin_outputs, "path": [i.split("/")[-1] for i in test_data]}).to_csv("../submission.csv", index=None)